In [8]:
import requests
from bs4 import BeautifulSoup
import pyodbc
import json
import time
import re

In [9]:
connection = pyodbc.connect('DRIVER={SQL SERVER};'
                            'SERVER=server_name;'
                            'DATABASE=IMDB;'
                            'UID=sa;'
                            'PWD=your_password')

cursor = connection.cursor()

In [10]:
cursor.execute('''
IF NOT EXISTS (SELECT * FROM sysobjects WHERE name = 'Most_Popular_TVShows' AND xtype = 'U')
CREATE TABLE Most_Popular_TVShows (
    ID INT PRIMARY KEY IDENTITY(1,1),
    SeriesTitle NVARCHAR(255),
    ReleaseDate NVARCHAR(50),
    Certificate NVARCHAR(50),
    Runtime NVARCHAR(50),
    ImdbRating NVARCHAR(50),
    Genre NVARCHAR(255),
    Overview TEXT,
    Director NVARCHAR(255),
    Stars NVARCHAR(MAX),
    NoOfVotes NVARCHAR(50),
    BudgetUSD MONEY,
    GrossWorldwideUSD MONEY,
    PosterLink NVARCHAR(MAX),
    TrailerLink NVARCHAR(MAX)
)
''')
connection.commit()

In [11]:
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/128.0.0.0 Safari/537.36'}
url = 'https://www.imdb.com/chart/tvmeter/?ref_=nv_tvv_mptv'
response = requests.get(url, headers=headers)
soup = BeautifulSoup(response.text, 'html.parser')

In [12]:
soup

<!DOCTYPE html>
<html lang="en-US" xmlns:fb="http://www.facebook.com/2008/fbml" xmlns:og="http://opengraphprotocol.org/schema/"><head><meta charset="utf-8"/><meta content="width=device-width" name="viewport"/><script>if(typeof uet === 'function'){ uet('bb', 'LoadTitle', {wb: 1}); }</script><script>window.addEventListener('load', (event) => {
        if (typeof window.csa !== 'undefined' && typeof window.csa === 'function') {
            var csaLatencyPlugin = window.csa('Content', {
                element: {
                    slotId: 'LoadTitle',
                    type: 'service-call'
                }
            });
            csaLatencyPlugin('mark', 'clickToBodyBegin', 1727769773696);
        }
    })</script><title>Most Popular TV Shows</title><meta content="As determined by IMDb users" data-id="main" name="description"/><script type="application/ld+json">{"@type":"ItemList","itemListElement":[{"@type":"ListItem","item":{"@type":"TVSeries","url":"https://www.imdb.com/title/t

In [13]:
json_data_tag = soup.find('script', type='application/ld+json')
json_data = json.loads(json_data_tag.string)
dramas = json_data['itemListElement']
for drama in dramas:
    drama_info = drama['item']
    genre = drama_info.get('genre', None)
    drama_url = drama_info.get('url', None)
    vote = drama_info.get('aggregateRating', {}).get('ratingCount', None)
    
    print(f'Genre: {genre}, URL: {drama_url}')

Genre: Biography, Crime, Drama, URL: https://www.imdb.com/title/tt13207736/
Genre: Crime, Drama, Fantasy, URL: https://www.imdb.com/title/tt15435876/
Genre: Action, Adventure, Drama, URL: https://www.imdb.com/title/tt7631058/
Genre: Action, Adventure, Comedy, URL: https://www.imdb.com/title/tt15571732/
Genre: Crime, Drama, Mystery, URL: https://www.imdb.com/title/tt11514868/
Genre: Drama, Horror, Mystery, URL: https://www.imdb.com/title/tt9813792/
Genre: Drama, Thriller, URL: https://www.imdb.com/title/tt5875444/
Genre: Crime, Drama, Mystery, URL: https://www.imdb.com/title/tt26748649/
Genre: Comedy, URL: https://www.imdb.com/title/tt26933824/
Genre: Action, Adventure, Drama, URL: https://www.imdb.com/title/tt0944947/
Genre: Comedy, Crime, Drama, URL: https://www.imdb.com/title/tt11691774/
Genre: Comedy, Drama, Fantasy, URL: https://www.imdb.com/title/tt8550732/
Genre: Crime, Drama, URL: https://www.imdb.com/title/tt16358384/
Genre: Drama, Horror, URL: https://www.imdb.com/title/tt3145

In [14]:
def drama_details(drama_url):
    drama_response = requests.get(drama_url, headers=headers)
    drama_soup = BeautifulSoup(drama_response.text, 'html.parser')

    details = {}

    title = drama_soup.find('h1', {'data-testid': 'hero__pageTitle'})
    if title:
        title_text = title.find('span', class_='hero__primary-text')
        details['title'] = title_text.text.strip() if title_text else None
    else:
        details['title'] = None

    release_date = drama_soup.find('li', {'data-testid': 'title-details-releasedate'})
    if release_date:
        release_date_text = release_date.find('a', class_='ipc-metadata-list-item__list-content-item ipc-metadata-list-item__list-content-item--link', href=True)
        details['release_date'] = release_date_text.text.strip() if release_date_text else None
    else:
        details['release_date'] = None

    certificate = drama_soup.find('ul', class_="ipc-inline-list ipc-inline-list--show-dividers sc-ec65ba05-2 joVhBE baseAlt")
    if certificate:
        a_tags = certificate.find_all('a')
        if len(a_tags) > 1:
            details['certificate'] = a_tags[1].text.strip()
        else:
            details['certificate'] = None
    else:
        details['certificate'] = None

    runtime = drama_soup.find('li', {'data-testid': 'title-techspec_runtime'})
    if runtime:
        runtime_text = runtime.find('div', class_='ipc-metadata-list-item__content-container')
        details['runtime'] = runtime_text.text.strip() if runtime_text else None
    else:
        details['runtime'] = None

    rating = drama_soup.find('div', {'data-testid': 'hero-rating-bar__aggregate-rating'})
    if rating:
        rating_text = rating.find('span', class_='sc-c4ffe080-1 iQZtLP')
        details['rating'] = rating_text.text.strip() if rating_text else None
    else:
        details['rating'] = None

    details['genre'] = genre
    
    description = drama_soup.find('p', {'data-testid': 'plot'})
    if description:
        description_text = description.find('span', class_='sc-9579cce5-1 ggovgq')
        details['description'] = description_text.text.strip() if description_text else None
    else:
        details['description'] = None

    director = drama_soup.find('li', {'data-testid': 'title-pc-principal-credit'})
    if director:
        director_text = director.find('a', class_='ipc-metadata-list-item__list-content-item ipc-metadata-list-item__list-content-item--link', href=True)
        details['director'] = director_text.text.strip() if director_text else None
    else:
        details['director'] = None
        
    star = drama_soup.find('div', {'class' : 'ipc-metadata-list-item__content-container'})
    if star:
        star_text = star.find_all('a', class_ = 'ipc-metadata-list-item__list-content-item ipc-metadata-list-item__list-content-item--link', href = True)
        stars = [a_tag.text.strip() for a_tag in star_text] 
        details['star'] = ', '.join(stars) if stars else None
    else:
        details['star'] = None

    details['votes'] = vote

    budget = drama_soup.find('li', {'data-testid': 'title-boxoffice-budget'})
    if budget:
        budget_text = budget.find('span', class_='ipc-metadata-list-item__list-content-item')
        details['budget'] = budget_text.text.strip() if budget_text else None
    else:
        details['budget'] = None

    gross = drama_soup.find('li', {'data-testid': 'title-boxoffice-cumulativeworldwidegross'})
    if gross:
        gross_text = gross.find('span', class_='ipc-metadata-list-item__list-content-item')
        details['gross'] = gross_text.text.strip() if gross_text else None
    else:
        details['gross'] = None

    poster = drama_soup.find('div', {'data-testid': 'hero-media__poster'})
    if poster:
        poster_link = poster.find('img', class_='ipc-image')
        details['poster'] = poster_link.get('src') if poster_link else None
    else:
        details['poster'] = None

    details['trailer'] = drama_url
    
    return details

def process_budget(details):
    budget_value = details['budget']
    if budget_value:
        budget_value = budget_value.replace('(estimated)', '').strip()
        match = re.match(r'([\D]+)\s*([\d,]+)', budget_value)
        if match:
            currency = match.group(1).strip()
            value_str = match.group(2)

            details['budget_currency'] = currency
            details['budget_value'] = value_str  # Store as string for conversion
        else:
            details['budget_currency'] = None
            details['budget_value'] = None
    else:
        details['budget_currency'] = None
        details['budget_value'] = None
        
def convert_to_usd(budget_value, budget_currency):
    
    currency_mapping = {
        'USD' : 1.0000,
        'CA$' : 0.74,
        '¥' : 0.0069,
        '€' : 1.11,
        '₹' : 0.012,
        '£' : 1.33,
        '₩' : 0.00076,
        'A$' : 0.69,
        "DEM" : 0.0026,
        "DKK" : 0.15,
        "R$" : 0.18
    }
    
    currency_rate = currency_mapping.get(budget_currency, 1.0000)
    
    numeric_value = float(budget_value.replace(',', ''))

    return numeric_value * currency_rate, 'USD'


for drama in dramas:
    drama_info = drama['item']
    title = drama_info.get('name', None)
    genre = drama_info.get('genre', None)
    drama_url = drama_info.get('url', None)
    vote = drama_info.get('aggregateRating', {}).get('ratingCount', None)
    
    print(f'Fetching details for {title} ({drama_url})...')
    
    try:
        drama_dtl = drama_details(drama_url)
            
        process_budget(drama_dtl)
        
        if drama_dtl['budget_value'] and drama_dtl['budget_currency']:
            converted_budget, budget_currency = convert_to_usd(drama_dtl['budget_value'], drama_dtl['budget_currency'])
        else:
            converted_budget, budget_currency = None, 'USD'
        
        cursor.execute('''
            INSERT INTO Most_Popular_TVShows 
            (SeriesTitle, ReleaseDate, Certificate, Runtime, ImdbRating, Genre, Overview, Director, Stars, NoOfVotes, BudgetUSD, GrossWorldwideUSD, PosterLink, TrailerLink) 
            VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
        ''', 
        drama_dtl['title'],
        drama_dtl['release_date'],
        drama_dtl['certificate'],
        drama_dtl['runtime'], 
        drama_dtl['rating'],
        drama_dtl['genre'],
        drama_dtl['description'], 
        drama_dtl['director'],
        drama_dtl['star'],
        drama_dtl['votes'],
        converted_budget,
        drama_dtl['gross'],
        drama_dtl['poster'], 
        drama_dtl['trailer'])
        
        connection.commit()
        
    except Exception as e:
        print(f"Error fetching details for {title}: {e}")
    
    time.sleep(2)
    
    
cursor.close()
connection.close()
    
#     print(f"Title: {drama_dtl['title']}")
#     print(f"Date: {drama_dtl['release_date']}")
#     print(f"Certificate: {drama_dtl['certificate']}")
#     print(f"Runtime: {drama_dtl['runtime']}")
#     print(f"Rating: {drama_dtl['rating']}")
#     print(f"Genre: {drama_dtl['genre']}")
#     print(f"Description: {drama_dtl['description']}")
#     print(f"Director: {drama_dtl['director']}")
#     print(f"Votes: {drama_dtl['votes']}")
#     print(f"Budget: {drama_dtl['budget']}")
#     print(f"Gross Worldwide: {drama_dtl['gross']}")
#     print(f"Poster Link: {drama_dtl['poster']}")
#     print(f"Trailer Link: {drama_dtl['trailer']}")


Fetching details for Monsters (https://www.imdb.com/title/tt13207736/)...
Fetching details for The Penguin (https://www.imdb.com/title/tt15435876/)...
Fetching details for The Lord of the Rings: The Rings of Power (https://www.imdb.com/title/tt7631058/)...
Fetching details for Agatha All Along (https://www.imdb.com/title/tt15571732/)...
Fetching details for The Perfect Couple (https://www.imdb.com/title/tt11514868/)...
Fetching details for From (https://www.imdb.com/title/tt9813792/)...
Fetching details for Slow Horses (https://www.imdb.com/title/tt5875444/)...
Fetching details for High Potential (https://www.imdb.com/title/tt26748649/)...
Fetching details for Nobody Wants This (https://www.imdb.com/title/tt26933824/)...
Fetching details for Game of Thrones (https://www.imdb.com/title/tt0944947/)...
Fetching details for Only Murders in the Building (https://www.imdb.com/title/tt11691774/)...
Fetching details for Kaos (https://www.imdb.com/title/tt8550732/)...
Fetching details for Tulsa